# References
- ["Exploring UPnP with Python" - Electric Monk](https://www.electricmonk.nl/log/2016/07/05/exploring-upnp-with-python/)
  - A lot of the SSDP code is a direct copy from this reference
- [upnpclient](https://github.com/flyte/upnpclient/blob/develop/upnpclient/soap.py)
  - A lot of the SOAP request code is a combination of this reference and the first reference

# Motivation
I just wanted to play around with UPNP device discovery. I'm sure you can find better code out there, but I do like the way I condensed down the Action and Argument definitions so it is really easy to see what is available.

# Terms
- **UPNP**: Universal Plug & Play
- **SSDP**: Simple Service Discovery Protocol
- **SCPD**: Service Control Point Definition
- **SOAP**: Simple Object Access Protocol

In [159]:
from upnp import UPNP_Explorer

### STEP 1: Broadcast

In [165]:
upnp = UPNP_Explorer() # Broadcast SSDP to UPNP Servers and Get Replies

In [166]:
upnp.SSDP.print_servers()

Samsung-Linux/4.1, UPnP/1.0, Samsung_UPnP_SDK/1.0_0
	http/1.1 200 ok: HTTP/1.1 200 OK
	cache-control: max-age=1800
	date: Wed, 15 Jul 2020 16:02:41 GMT
	location: http://192.168.1.14:9197/dmr
	st: upnp:rootdevice
	usn: uuid:036cb031-f5f8-4831-bcd7-c0259ee25740::upnp:rootdevice
	content-length: 0
	bootid.upnp.org: 4

Samsung-Linux/4.1, UPnP/1.0, Samsung_UPnP_SDK/1.0_1
	http/1.1 200 ok: HTTP/1.1 200 OK
	cache-control: max-age=1800
	date: Wed, 15 Jul 2020 16:02:41 GMT
	location: http://192.168.1.14:7678/nservice/
	st: upnp:rootdevice
	usn: uuid:f4ec0a2d-c0aa-4f9d-904a-ecc457202ba9::upnp:rootdevice
	content-length: 0
	bootid.upnp.org: 5

Roku/9.3.0 UPnP/1.0 Roku/9.3.0_0
	http/1.1 200 ok: HTTP/1.1 200 OK
	cache-control: max-age=3600
	st: upnp:rootdevice
	usn: uuid:02706106-8001-10a3-80b0-b0a7374b9165::upnp:rootdevice
	location: http://192.168.1.3:8060/
	wakeup: MAC=b0:a7:37:4b:91:65;Timeout=10



### STEP 2: Define Server
- Gets Root XML using Location URL

In [167]:
SEARCH_SERVER_NAME = 'samsung'
url_list = upnp.define_server(SEARCH_SERVER_NAME)

print('Location URL: ', upnp.SCPD.location_url) # Contains Device Definition and Service List

Multiple servers found for server name (samsung). Please use server_idx input argument to specify which server to use or be more specific when specifying server_name. 0 index currently used.
Servers found:
 	Samsung-Linux/4.1, UPnP/1.0, Samsung_UPnP_SDK/1.0_0
	Samsung-Linux/4.1, UPnP/1.0, Samsung_UPnP_SDK/1.0_1

Location URL:  http://192.168.1.14:9197/dmr


In [168]:
# Pretty Print SCPD Root XML
upnp.print_element_pretty(upnp.SCPD.root)

specVersion--> 
	major--> 1
	minor--> 0
device--> 
	deviceType--> urn:schemas-upnp-org:device:MediaRenderer:1
	X_compatibleId--> MS_DigitalMediaDeviceClass_DMR_V001
	X_deviceCategory--> Display.TV.LCD Multimedia.DMR
	X_DLNADOC--> DMR-1.50
	friendlyName--> [TV] Samsung 6 Series (75)
	manufacturer--> Samsung Electronics
	manufacturerURL--> http://www.samsung.com/sec
	modelDescription--> Samsung TV DMR
	modelName--> UN75NU6950
	modelNumber--> AllShare1.0
	modelURL--> http://www.samsung.com/sec
	serialNumber--> 07ZX3CDN101794W
	UDN--> uuid:036cb031-f5f8-4831-bcd7-c0259ee25740
	iconList--> 
		icon--> 
			mimetype--> image/jpeg
			width--> 48
			height--> 48
			depth--> 24
			url--> /icon_SML.jpg
		icon--> 
			mimetype--> image/jpeg
			width--> 120
			height--> 120
			depth--> 24
			url--> /icon_LRG.jpg
		icon--> 
			mimetype--> image/png
			width--> 48
			height--> 48
			depth--> 24
			url--> /icon_SML.png
		icon--> 
			mimetype--> image/png
			width--> 120
			height--> 120
			depth--> 24
	

### STEP 3a: Choose URL Set
- Choose Control URL (SOAP Commands sent to this URL)
- Choose SCPD URL (Service URL with Argument & Action Definition XML)
- Choose Type (Define for SOAP Requests)

In [169]:
for i, urls in enumerate(upnp.SCPD.url_list):
    print('---- URL Set Index %s ----'%(i))
    for k, v in urls.items():
        print('%s: %s'%(k,v))
    print()

---- URL Set Index 0 ----
ctrl: http://192.168.1.14:9197/upnp/control/RenderingControl1
scpd: http://192.168.1.14:9197/RenderingControl_1.xml
type: urn:schemas-upnp-org:service:RenderingControl:1

---- URL Set Index 1 ----
ctrl: http://192.168.1.14:9197/upnp/control/ConnectionManager1
scpd: http://192.168.1.14:9197/ConnectionManager_1.xml
type: urn:schemas-upnp-org:service:ConnectionManager:1

---- URL Set Index 2 ----
ctrl: http://192.168.1.14:9197/upnp/control/AVTransport1
scpd: http://192.168.1.14:9197/AVTransport_1.xml
type: urn:schemas-upnp-org:service:AVTransport:1



In [170]:
# Choose Index from above list
USE_IDX = 0

CTRL_URL = upnp.SCPD.url_list[USE_IDX]['ctrl']
SCPD_URL = upnp.SCPD.url_list[USE_IDX]['scpd']
SERVICE_TYPE = upnp.SCPD.url_list[USE_IDX]['type']
print(CTRL_URL)
print(SCPD_URL)
print(SERVICE_TYPE)

http://192.168.1.14:9197/upnp/control/RenderingControl1
http://192.168.1.14:9197/RenderingControl_1.xml
urn:schemas-upnp-org:service:RenderingControl:1


### STEP 3b: Get Service XML
- Gets the Service XML using the SCPD_URL
- Service XML contains Action & Argument Definitions

In [171]:
upnp.SCPD.get_service_xml(SCPD_URL)

# Print Service XML
upnp.print_element_pretty(upnp.SCPD.service_root)

specVersion--> 
	major--> 1
	minor--> 0
actionList--> 
	action--> 
		name--> ListPresets
		argumentList--> 
			argument--> 
				name--> InstanceID
				direction--> in
				relatedStateVariable--> A_ARG_TYPE_InstanceID
			argument--> 
				name--> CurrentPresetNameList
				direction--> out
				relatedStateVariable--> PresetNameList
	action--> 
		name--> SelectPreset
		argumentList--> 
			argument--> 
				name--> InstanceID
				direction--> in
				relatedStateVariable--> A_ARG_TYPE_InstanceID
			argument--> 
				name--> PresetName
				direction--> in
				relatedStateVariable--> A_ARG_TYPE_PresetName
	action--> 
		name--> GetMute
		argumentList--> 
			argument--> 
				name--> InstanceID
				direction--> in
				relatedStateVariable--> A_ARG_TYPE_InstanceID
			argument--> 
				name--> Channel
				direction--> in
				relatedStateVariable--> A_ARG_TYPE_Channel
			argument--> 
				name--> CurrentMute
				direction--> out
				relatedStateVariable--> Mute
	action--> 
		name--> SetMute
		argumentLi

# SOAP Preparation

### Step 4a: Define Action used and Action's Input Arguments

In [176]:
upnp.SCPD.pretty_print_actions()

ListPresets(InstanceID)
	Returns: (CurrentPresetNameList)
SelectPreset(InstanceID, PresetName)
GetMute(InstanceID, Channel)
	Returns: (CurrentMute)
SetMute(InstanceID, Channel, DesiredMute)
GetVolume(InstanceID, Channel)
	Returns: (CurrentVolume)
SetVolume(InstanceID, Channel, DesiredVolume)
X_GetAspectRatio(InstanceID)
	Returns: (AspectRatio)
X_SetAspectRatio(InstanceID, AspectRatio)
X_Move360View(InstanceID, LatitudeOffset, LongitudeOffset)
X_Zoom360View(InstanceID, ScaleFactorOffset)
X_Origin360View(InstanceID)
X_ControlCaption(InstanceID, Operation, Name, ResourceURI, CaptionURI, CaptionType, Language, Encoding)
X_GetCaptionState(InstanceID)
	Returns: (Captions, EnabledCaptions)
X_GetServiceCapabilities(InstanceID)
	Returns: (ServiceCapabilities)
X_SetZoom(InstanceID, x, y, w, h)
X_GetTVSlideShow(InstanceID)
	Returns: (CurrentShowState, CurrentThemeId, TotalThemeNumber)
X_SetTVSlideShow(InstanceID, CurrentShowState, CurrentShowTheme)


In [177]:
ACTION_NAME = 'SetMute'

In [178]:
upnp.SCPD.print_action_arg_desc(ACTION_NAME)

_______input arguments________
InstanceID
	dataType: ui4
Channel
	dataType: string
	allowedValueList: Master
DesiredMute
	dataType: boolean

_______output arguments________



In [179]:
argument_inputs = upnp.SCPD.define_action(ACTION_NAME)

print(upnp.SCPD.input_args)

{'InstanceID': 0, 'Channel': 'Master', 'DesiredMute': None}


### Step 4b: Change any input action values if necessary or desired

In [184]:
# Modify any arguments you would like to manually change
argument_inputs['DesiredMute'] = 0

print(upnp.SCPD.input_args)

{'InstanceID': 0, 'Channel': 'Master', 'DesiredMute': 0}


### Step 5: SOAP Call

In [185]:
upnp.request_action(CTRL_URL, SERVICE_TYPE) # ACTION_NAME & INPUT_ARGS defined from SCPD.define_action

# OR if ACTION_NAME & INPUT_ARGS are defined elsewhere

# upnp.request_action(CTRL_URL, SERVICE_TYPE, ACTION_NAME, INPUT_ARGS)

In [186]:
upnp.print_xml(upnp.SOAP.soap_body)

<?xml version="1.0" ?>
<SOAP-ENV:Envelope SOAP-ENV:encodingStyle="http://schemas.xmlsoap.org/soap/encoding/" xmlns:SOAP-ENV="http://schemas.xmlsoap.org/soap/envelope">
	<SOAP-ENV:Body>
		<m:SetMute xmlns:m="urn:schemas-upnp-org:service:RenderingControl:1">
			<InstanceID>0</InstanceID>
			<Channel>Master</Channel>
			<DesiredMute>0</DesiredMute>
		</m:SetMute>
	</SOAP-ENV:Body>
</SOAP-ENV:Envelope>



In [187]:
upnp.print_xml(upnp.SOAP.response)

<?xml version="1.0" ?>
<s:Envelope s:encodingStyle="http://schemas.xmlsoap.org/soap/encoding/" xmlns:s="http://schemas.xmlsoap.org/soap/envelope/">
	<s:Body>
		<u:SetMuteResponse xmlns:u="urn:schemas-upnp-org:service:RenderingControl:1"/>
	</s:Body>
</s:Envelope>

